In [113]:
# font 설정
import platform
path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else :
    print('Unknown system... sorry~')
#########################################################################
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


https://injo.tistory.com/31  
부스팅 알고리즘  
https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-15-Gradient-Boost

In [114]:
train =  pd.read_csv("bike/train.csv",parse_dates=["datetime"])
test = pd.read_csv("bike/test.csv",parse_dates=["datetime"])

In [115]:
train['year'] =train['datetime'].dt.year
train['month'] =train['datetime'].dt.month
train['hour'] =train['datetime'].dt.hour
train['dayofweek'] =train['datetime'].dt.dayofweek

test['year'] =test['datetime'].dt.year
test['month'] =test['datetime'].dt.month
test['hour'] =test['datetime'].dt.hour
test['dayofweek'] =test['datetime'].dt.dayofweek

In [116]:
#이상치 제거
train_out =  train[np.abs(train['count']-train['count'].mean())<=(3*train['count'].std())]

In [117]:
print(train.shape)
print(train_out.shape)
print(test.shape)

(10886, 16)
(10739, 16)
(6493, 13)


In [121]:
train_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10739 entries, 0 to 10885
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10739 non-null  datetime64[ns]
 1   season      10739 non-null  int64         
 2   holiday     10739 non-null  int64         
 3   workingday  10739 non-null  int64         
 4   weather     10739 non-null  int64         
 5   temp        10739 non-null  float64       
 6   atemp       10739 non-null  float64       
 7   humidity    10739 non-null  int64         
 8   windspeed   10739 non-null  float64       
 9   casual      10739 non-null  int64         
 10  registered  10739 non-null  int64         
 11  count       10739 non-null  int64         
 12  year        10739 non-null  int64         
 13  month       10739 non-null  int64         
 14  hour        10739 non-null  int64         
 15  dayofweek   10739 non-null  int64         
dtypes: datetime64[ns](1), 

In [81]:
#windspeed 값 채우기
# train =  pd.read_csv("bike/train.csv",parse_dates=["datetime"])
# test = pd.read_csv("bike/test.csv",parse_dates=["datetime"])

In [122]:
#train, test 합치기
data = train_out.append(test)
data.reset_index(inplace= True)
data.drop('index', inplace=True, axis=1)

In [123]:
data['year'] =data['datetime'].dt.year
data['month'] =data['datetime'].dt.month
data['hour'] =data['datetime'].dt.hour
data['dayofweek'] =data['datetime'].dt.dayofweek

In [84]:
#windspeed 채우기
dataWind0 = data[data['windspeed']==0]
dataWind1 = data[data['windspeed']!=0]
#모델 객체 생성, 사용할 feature(windCol), fit하기 
rfModel_wind = RandomForestRegressor()
windCol= ['season', 'weather','humidity', 'month', 'year', 'temp','atemp']
rfModel_wind.fit(dataWind1[windCol], dataWind1['windspeed'])
#예측, 대입 
wind0Values = rfModel_wind.predict(X = dataWind0[windCol])
dataWind0['windspeed']=wind0Values

data = dataWind1.append(dataWind0)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)

C:\Users\shami\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [124]:
dropFeatures = ['casual',"count","registered"]

In [125]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=["datetime"])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=["datetime"])
datatime_=dataTest["datetime"]

In [126]:
label = dataTrain['count']
xData = dataTrain.drop(['datetime','casual','registered','count'],axis=1)
test = dataTest.drop(['datetime','casual','registered','count'],axis=1)

In [127]:
rfModel = GradientBoostingRegressor()
cv=RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(rfModel, xData, label, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
rfModel.fit(xData, label)
prediction= rfModel.predict(test)

In [128]:
res=pd.DataFrame({'datetime':datatime_,'count':prediction})

In [129]:
res.loc[res['count']<0, 'count'] = 0

In [130]:
res

,datetime,count
10739,2011-01-20 00:00:00,2.654379
10740,2011-01-20 01:00:00,0.000000
10741,2011-01-20 02:00:00,0.000000
10742,2011-01-20 03:00:00,0.000000
10743,2011-01-20 04:00:00,0.000000
...,...,...
17227,2012-12-31 19:00:00,315.421826
17228,2012-12-31 20:00:00,216.462519
17229,2012-12-31 21:00:00,171.153143
17230,2012-12-31 22:00:00,131.807433


In [ ]:

# rfModel = RandomForestRegressor()
# rfModel.fit(xData, label)
# prediction = rfModel.predict(test)
# # res = pd.DataFrame({'datetime':datetime_,'count':prediction})

In [131]:
res.to_csv("result4.csv",index=False)
pd.read_csv("result4.csv")

,datetime,count
0,2011-01-20 00:00:00,2.654379
1,2011-01-20 01:00:00,0.000000
2,2011-01-20 02:00:00,0.000000
3,2011-01-20 03:00:00,0.000000
4,2011-01-20 04:00:00,0.000000
...,...,...
6488,2012-12-31 19:00:00,315.421826
6489,2012-12-31 20:00:00,216.462519
6490,2012-12-31 21:00:00,171.153143
6491,2012-12-31 22:00:00,131.807433


result1 : 수업시간에 했던 내용에 GradientBoostingRegressor()로 마무리  
result2 : windspeed도 GBM으로 채움  
3: 파라미터 지정